### Simple EDA

간단하게 데이터를 탐색한다.

In [ ]:
import pandas as pd

# 컬럼이 없는 데이터 프레임들은 임의로 컬럼명을 생성하여 사용한다.
df1 = pd.DataFrame(pd.read_csv("./crawl_14435163.csv").values, columns=['title', 'write_date', 'views']) # 데이터프레임 1
df2 = pd.DataFrame(pd.read_csv("./crawl_20720299.csv").values, columns=['title', 'write_date', 'views']) # 데이터프레임 2

df1 = df1[['title', 'views']] # title과 views열만 사용한다
df2 = df2[['title', 'views']]
df3 = pd.read_csv("./naver_cafe_crawl_fix.csv")[['title', 'views']]

### about df1

In [ ]:
df1.isna().sum() # 결측치 확인

In [ ]:
df1.info() # data type 확인

In [ ]:
df1.dropna(inplace=True, axis = 0) # 결측치 제거

df1['title'] = df1['title'].str.replace("[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-zA-Z ]", "", regex=True) # 한글과 알파벳, 공백을 제외하고 제거
df1['views'] = df1['views'].str.replace("[^0-9]", "", regex=True).astype(int) # label값을 구하기 위해 int타입으로 형전환

dup_cnt = df1['title'].loc[df1['title'].duplicated() == True].count() # 중복되는 개수 조회
print(f"{dup_cnt}개의 중복 자료가 존재합니다.")

df1.drop_duplicates(subset=['title'], inplace=True) # 중복 행 제거
df1 = df1.reset_index(drop = True) # 인덱스 초기화
print(f"{df1.isna().sum().sum()}개의 결측치가 존재합니다.") # 결측치 확인

In [ ]:
df1.head() # 한글과 알파엣, 공백이 잘 지워졌는지 확인

### about df2

In [ ]:
df2.isna().sum() # 결측치 확인

In [ ]:
df2.info() # data type 확인

In [ ]:
# df1과 동일한 방식으로 진행
df2.dropna(inplace=True, axis = 0)

df2['title'] = df2['title'].str.replace("[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-zA-Z ]", "", regex=True)
df2['views'] = df2['views'].str.replace("[^0-9]", "", regex=True).astype(int)

dup_cnt = df2['title'].loc[df2['title'].duplicated() == True].count()
print(f"{dup_cnt}개의 중복 자료가 존재합니다.")

df2.drop_duplicates(subset=['title'], inplace=True)
df2 = df2.reset_index(drop = True)
print(f"{df2.isna().sum().sum()}개의 결측치가 존재합니다.")

### about df3

In [ ]:
df3.isna().sum() # 결측치 확인

In [ ]:
df3.info() # data type 확인

In [ ]:
# 소수점 자리 아래 확인을 위해서 views 컬럼을 string으로 타입캐스팅
df3['views'] = df3['views'].astype('string')
df3['views'].loc[df3['views'].str.endswith(".0") == True].count() # 모두 소수점 아래 자리는 0이다.

In [ ]:
df3.dropna(inplace=True, axis = 0) # 결측치 제거

In [ ]:
# 소수점 아래 자리가 모두 .0 이므로 차후에 정수형으로 변환한다.
df3['views'] = df3['views'].astype('string')
df3['views'][0].endswith(".0")

In [ ]:
#  df1, df2에서 진행한 전처리과정 동일하게 진행
df3['title'] = df3['title'].str.replace("[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-zA-Z ]", "", regex=True)
df3['views'] = df3['views'].str.replace("[^0-9].", "", regex=True).astype(float).astype(int)

dup_cnt = df3['title'].loc[df3['title'].duplicated() == True].count()
print(f"{dup_cnt}개의 중복 자료가 존재합니다.")

df3.drop_duplicates(subset=['title'], inplace=True)
df3 = df3.reset_index(drop = True)
print(f"{df3.isna().sum().sum()}개의 결측치가 존재합니다.")

In [ ]:
df = pd.concat([df1, df2, df3]).reset_index(drop = True) # 전처리 완료한 세 개의 데이터 프레임을 합친다.

In [ ]:
df.isna().sum() # 결측치 다시확인

In [ ]:
df.head()

In [ ]:
df.info() # data type 확인

### 라벨링 진행
각 view_cnt의 평균을 내서 평균보다 높으면 1, 아니라면 0을 부여한다.

In [ ]:
mean_val = int(df['views'].mean())

In [ ]:
mean_val

In [ ]:
def get_label_list(values, standard):
    label_list = []
    for value in values:
        if value >= standard:
            label_list.append(1)
        else:
            label_list.append(0)
            
    return label_list

df['label'] = get_label_list(df['views'].values, mean_val)

### 데이터 병합
전처리 후 라벨링 한 두 데이터 프레임을 병합합니다.

In [ ]:
df

### 라벨값 분포 확인
각 라벨별 분포를 확인합니다.

In [ ]:
df['label'].value_counts()

### 시행착오 1 : 라벨값 간 불균형

평균

|라벨 0|라벨 1|
|---|---|
|46544건|25657건|

평균은 라벨값 간의 큰 격차를 불러오기에, 분위수를 사용한다

제 1 사분위수

|라벨 0|라벨 1|
|---|---|
|18040건|54161건|

제 2 사분위수

|라벨 0|라벨 1|
|---|---|
|36001건|36200건|

그나마 균등하게 분포되는 제 2 사분위수로 채택

In [ ]:
df_quantile1 = df['views'].quantile(.5)
print(int(df_quantile1))

In [ ]:
df['label'] = get_label_list(df['views'], df_quantile1)

In [ ]:
df

In [ ]:
df['label'].value_counts()
# df.to_csv("./origin_df.csv", sep=",", encoding="utf8", index=False)

In [ ]:
df['views'].mean()

In [ ]:
pd.read_csv("./origin_df.csv")